In [ ]:
!pip install face_recognition
!pip install click
!pip install schedule
!pip install timeloop
!pip install opencv-python
!pip install opencv-contrib-python

In [9]:
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)
import face_recognition
import numpy as np
import cv2
import pandas as pd
import os
from google.colab.patches import cv2_imshow
import os
import PIL
import io
from base64 import b64decode, b64encode
import time
from datetime import datetime
import xlrd
import pandas as pd
import time
from datetime import date
import csv
today = date.today().strftime('%Y-%m-%d %H:%M:%S')
os.chdir('/content/drive/MyDrive/attendance')
import Video_Capture
capture_number = 0
import click
import schedule
import time
import datetime
import pytz



Mounted at /content/drive/


In [10]:
50551# Set the timezone to your desired timezone
tz = pytz.timezone('America/New_York')

e = datetime.datetime.now(tz)
print(e)
start_hour =17
strat_minute = 48
finish_hour = 17
finish_minute = 49
time_to_capture = 30 # waiting time for capture next photo (second)

2023-04-05 17:48:53.395009-04:00


In [11]:
def streaming_video(e):
    a = Video_Capture
    capture_number = 1
    video_stream = a.Video_Capture(0).video_stream
    video_frame = a.Video_Capture(0).video_frame
    js_to_image = a.Video_Capture(0).js_to_image
    bbox_to_bytes = a.Video_Capture(0).bbox_to_bytes

    video_stream()
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    

    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break
        # convert JS response to OpenCV Image
        image_bytes = b64decode(js_reply["img"].split(',')[1])
        #print(js_reply["img"])
        #img = js_to_image(js_reply["img"])
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        # decode numpy array into OpenCV BGR image
        img = cv2.imdecode(jpg_as_np, flags=1)
        # create transparent overlay for bounding box
        bbox_array = np.zeros([600,600,4], dtype=np.uint8)

        # grayscale image for face detection
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        os.chdir('/content/drive/MyDrive/attendance/test/')# change the path
        filename = f"{e.hour}:{e.minute}:{e.second}.png"
        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        # convert overlay of bbox into bytes
        bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
        #iobuf = io.BytesIO()
        # format bbox into png for return
        bbox_PIL.save(filename)
        # format return string
        #bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
          # update bbox so next frame gets new overlay
        #bbox = bbox_bytes
        ### inja ##########################
        cv2.imwrite(filename, img)
        capture_number += 1
        cv2.destroyAllWindows()
        break
    return True

In [12]:
count = 0
while True:
  if e.hour >= start_hour and e.hour <= finish_hour and e.minute >= strat_minute and e.minute <= finish_minute:
    print('Its time for rollcall')
    streaming_video(e)
    print('Image Captured')
    time.sleep(time_to_capture)
    e = datetime.datetime.now(tz)
    count += 1

    e = datetime.datetime.now(tz)
  else:
    e = datetime.datetime.now(tz)
    print("The hour is out of rollcall time ")
    break


Its time for rollcall


<IPython.core.display.Javascript object>

Image Captured
Its time for rollcall


<IPython.core.display.Javascript object>

Image Captured
The hour is out of rollcall time 


In [13]:
def read_train_images(folder , dim):
  X = []
  encode = []
  y = []
  Y = []
  n_samples=0
  none_image = 0
  for i in os.listdir(folder):
      #print(i) 
      img_file = cv2.imread( folder +'/' + i)
      #cv2_imshow(img_file)
      if img_file is not None:
          img_orginal=cv2.resize(img_file,dim)
          face_location = face_recognition.face_locations(img_orginal)
          encoded_img = face_recognition.face_encodings(img_orginal, face_location)
          #print(face_location)            
          X.append(img_orginal)
          encode.append(encoded_img)
          name = os.path.split(i)[-1]
          Y.append(name)
          n_samples+=1    
      else:
        none_image=+1
        print(f'there is {none_image} "not image" file in this folder')

  return X , Y, encode
dim = (600,600)
train_folder =  "/content/drive/MyDrive/attendance/train"
x , y , encoded_images= read_train_images(train_folder, dim)

In [14]:
def read_test_image(test_folder, dim, encoded_images, y):
  Attendees_list = []
  k=0
  for iter in os.listdir(test_folder):
    test_img = cv2.imread(test_folder + iter)
    j = 0
    for i in encoded_images:
      test_img=cv2.resize(test_img,dim)
      face_location_test = face_recognition.face_locations(test_img)
      encoded_test = face_recognition.face_encodings(test_img, face_location_test)
      #print(len(encoded_test))
      if len(encoded_test) == 0:
        results = []
        
      else:
        results = face_recognition.compare_faces(np.array(i), np.array(encoded_test))
        print(results)
        if len(results) == 0:
          print('No face were detected')
        else:
          if results[0]:
            Attendees_list.append(y[j])
          else:
            pass
      j+=1
    k+=1   
  return Attendees_list, k
test_folder = "/content/drive/MyDrive/attendance/test/"
raw_list, K = read_test_image(test_folder, dim, encoded_images, y)


[False]
[True]
[]
No face were detected
[True]
[]
No face were detected
[False]
[]
No face were detected
[]
No face were detected


In [15]:
lis_of_attendees = []
[lis_of_attendees.append(x) for x in raw_list if x not in lis_of_attendees]

absent_list = list(set(y).difference(lis_of_attendees))
Absents = []
for i in absent_list:
  name = i.split('.')[0]
  Absents.append(name)

Attendees = []
for i in lis_of_attendees:
  name = i.split('.')[0]
  Attendees.append(name)
print(f'Attendees students are :{Attendees}')
print(f'Absents students are :{Absents}')

if K>len(raw_list):
  strange_no = K-len(raw_list)
 # print(f'There are {strange_no} strange person in the class. Be careful!!')
  os.chdir('/content/drive/MyDrive/attendance/')


Attendees students are :['Sherea', 'Yesha']
Absents students are :['jack', 'Triza', 'sara', 'John', 'brad', 'Amin']


In [16]:
#making report
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment,Border,Side
from openpyxl import Workbook, load_workbook


#make time and date dynamic

# Define the data as lists
class_name = "A"
#time = "5:15 to 5:20"
#date = "2023-03-27"
teacher_name='Rachel Green'

date = e.strftime("%Y-%m-%d")
time = f"{start_hour:02d}:{strat_minute:02d} to {finish_hour:02d}:{finish_minute:02d}"

numbers = [1, 2, 3, 4, 5, 6, 7, 8]
#names = ["amin", "yesha", "triza", "sherea", "jack", "john", "sara", "soozan"]
student_ids = [605, 608, 604, 606, 609, 602, 603, 601]
#statuses = ["absent", "absent", "absent", "absent", "absent", "absent", "absent", "absent"]


# Combine attendance and absent lists
names = lis_of_attendees + absent_list


 #Create a list of statuses
statuses = []
for person in names:
    if person in lis_of_attendees:
        statuses.append("present")
    else:
        statuses.append("absent")



# Remove file extensions from names
names = [os.path.splitext(name)[0] for name in names]
# Open the Excel file
#workbook = openpyxl.Workbook()

# Select the worksheet you want to modify
#worksheet = workbook.active

In [17]:
import openpyxl
import os
from openpyxl.styles import PatternFill, Font, Alignment,Border,Side
from openpyxl.styles import PatternFill

workbook =  load_workbook(filename="classB.xlsx")

worksheet = workbook.active

# Define the data as lists
class_name = "A"
#time = "8 to 9"
#date = "2023-03-24"
teacher_name='Rachel Green'

numbers = [1, 2, 3, 4, 5, 6, 7, 8]
#names = ["Amin", "Yesha", "Triza", "Sherea", "Jack", "John", "Sara", "Soozan"]
student_ids = [100851605, 100851608, 100851604, 100851606, 100851609, 100851602, 100851603, 100851601]
#statuses = ["present", "present", "absent", "present", "present", "present", "present", "present"]

# Open the Excel file
workbook = openpyxl.Workbook()

# Select the worksheet you want to modify
worksheet = workbook.active



# worksheet.merge_cells
worksheet.merge_cells('A1:E2')

merged_cell = worksheet['A1']
merged_cell.alignment = Alignment(horizontal='center', vertical='center')
merged_cell.font = Font(bold=True, size=17)



            
# Create a new font with bold and increased size properties
bold_large_font = Font(bold=True, size=12)
# Write the data to specific cells
worksheet['A1'] = 'Class ' + class_name



worksheet['A3']='Teacher\'s Name'
worksheet.column_dimensions['A'].width = 15
worksheet['A3'].font = bold_large_font
worksheet.column_dimensions['B'].width = 12
worksheet['B3'] =  teacher_name

worksheet['C3'] = 'Time '
worksheet['C3'].font = bold_large_font
worksheet.column_dimensions['C'].width = 12
worksheet['D3'] =  time


worksheet['E3'] = 'Date: '+date
worksheet['E3'].font = bold_large_font
worksheet.column_dimensions['E'].width = 10

# Define the fill color
grey_fill = PatternFill(start_color='D3D3D3', end_color='DCDCDC', fill_type='solid')

# Loop through the cells A4 to F4 and set their fill color
for col in range(1, 6):
    cell = worksheet.cell(row=4, column=col)
    cell.fill = grey_fill
    

    
# Define the border style
border_style = Border(
    left=Side(border_style='thick'),
    right=Side(border_style='thick'),
    top=Side(border_style='thick'),
    bottom=Side(border_style='thick')
)


worksheet.row_dimensions[1].height = 20
worksheet.row_dimensions[2].height = 20
worksheet.row_dimensions[3].height = 25



# Set the cell's font to the newly created font
worksheet['A5'] = 'Number'
worksheet.column_dimensions['A'].width = 16
worksheet['A5'].font = bold_large_font
worksheet['A5'].alignment = Alignment(horizontal='center')



worksheet['B5'] = 'Name'
worksheet.column_dimensions['B'].width = 14
worksheet['B5'].font = bold_large_font
worksheet['B5'].alignment = Alignment(horizontal='center')



worksheet['C5'] = 'Student id'
worksheet.column_dimensions['C'].width = 15
worksheet['C5'].font = bold_large_font
worksheet['C5'].alignment = Alignment(horizontal='center')



worksheet['D5'] = 'Status'
worksheet.column_dimensions['D'].width = 14
worksheet['D5'].font = bold_large_font
worksheet['D5'].alignment = Alignment(horizontal='center')



worksheet.column_dimensions['E'].width = 18





# Define fill colors for status field
present_fill = PatternFill(fill_type='solid', start_color='CEE4CE', end_color='CEE4CE')
absent_fill = PatternFill(fill_type='solid', start_color='FACACA', end_color='FACACA')


#Set the thick border style (Bottom)

for i in range(len(numbers)):
    row = i + 6
    worksheet.row_dimensions[row].height = 20
    worksheet.cell(row=row, column=1, value=numbers[i]).alignment = Alignment(horizontal='center')
    worksheet.cell(row=row, column=2, value=names[i]).alignment = Alignment(horizontal='center')
    worksheet.cell(row=row, column=3, value=student_ids[i]).alignment = Alignment(horizontal='center')
    worksheet.cell(row=row, column=4, value=statuses[i]).alignment = Alignment(horizontal='center')
   


    
    # Set the fill color based on the value of the "status" field
    if statuses[i] == 'present':
        worksheet.cell(row=row, column=4).fill = present_fill
        
    else:
        worksheet.cell(row=row, column=4).fill = absent_fill


# get the last row and column of data
last_row = worksheet.max_row
last_column = worksheet.max_column

# loop over all cells from A1 to the last cell and add borders
for row in range(1, last_row + 1):
    for column in range(1, last_column + 1):
        cell = worksheet.cell(row=row, column=column)
        border = Border(left=Side(style='thick'), 
                        right=Side(style='thick'), 
                        top=Side(style='thick'), 
                        bottom=Side(style='thick'))
        cell.border = border
        


# Save the changes to the Excel file
workbook.save('classB.xlsx')


In [18]:
#mailing report to teacher
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
from google.colab import drive



# Set email parameters
sender = 'durhamattendance@gmail.com'
password = 'ytsfrxfvssiaucms'
recipient = 'muthonikiilu28@gmail.com'
subject = 'Test Email with Excel attachment'
body = 'Please find attached the Excel file.'

# Set file parameters
filename = 'classB.xlsx'
filepath = '/content/drive/MyDrive/attendance/' + filename

# Create email message
msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = recipient
msg['Subject'] = subject
msg.attach(MIMEText(body))

# Attach file to email message
with open(filepath, "rb") as attachment:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename= {filename}")
    msg.attach(part)

# Send email using SMTP server
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender, password)
    server.sendmail(sender, [recipient], msg.as_string())
    server.quit()
    print(f'Success: Email sent to {recipient} with attachment {filename}.')
except Exception as e:
    print(f'Error: Unable to send email to {recipient}.')
    print(e)


Success: Email sent to muthonikiilu28@gmail.com with attachment classB.xlsx.
